In [1]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import requests
import numpy as np
import pandas as pd
import json

In [ ]:
def get_tutor_info(zip_code, links, headers):
    
    mains = []
    scheds = []
    empty = {'sunday': [None], 'monday': [None], 'tuesday': [None], 'wednesday': [None],
         'thursday': [None], 'friday': [None], 'saturday': [None]}
    
    for i, link in enumerate(links):
        r = requests.get(link, headers = headers).text
        soup = BeautifulSoup(r, 'html.parser')

        tags = soup.find_all('script')
        time_tutoring = soup.find_all('h3')[0].text
        bio_list = soup.find_all('p', {'class':  'spc-zero-n spc-sm-s'})
        bio = [part.text for part in bio_list]
        if soup.find_all('i' ,{'class':'wc-background-pass wc-green'}):
            bck = 1
        else:
            bck = 0
        
        main_details = json.loads(tags[0].string)
        main_details['time_tutoring'] = time_tutoring
        main_details['bio'] = bio
        main_details['background_check'] = bck
        
        mains.append(main_details)
        try:
            scheds.append(json.loads(tags[6].string.split('=')[1][:-1]))
        except:
            scheds.append(empty)
        time.sleep(np.random.rand()*5)
        
    df = pd.merge(pd.DataFrame(mains), pd.DataFrame(scheds), left_index=True, right_index=True)
    df.to_csv('/home/codonnell/tutors_' + str(zip_code) + '.csv')

In [ ]:
search_add = 'https://www.wyzant.com/match/search?sort=1&d=20&utc_offset=-5&min_price=10&max_price=200&min_age=18&max_age=100&gender_pref=none&st=5&kw=physics&ol=false&z='
zip_code = 78750
url = search_add + str(zip_code)

chromedriver = '/home/codonnell/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)
driver.get(url)

#run this, then manually skip fields on the website to get to search page
#then run next cell
#search for all tutor links on page

In [2]:
#list of zip codes and subjects to search

zips = [91911,37128,85001]
subjects = ['physics', 'math', 'spanish', 'english']

#types of sorts to perform on webpage, 1: best match, 2: price highest-lowest, 3: price lowest-highest
sorts = [1, 2, 3]

for zip_code in zips:
    
    links = []
    tot_tuts = []
    
    for subject in subjects:
        
        
        for sort in sorts:
            
            url = 'https://www.wyzant.com/match/search?sort=' + str(sort) + '&d=20&utc_offset=-5&min_price=10&max_price=200&min_age=18&max_age=100&gender_pref=none&st=0&kw=' + subject + '&ol=false&z=' + str(zip_code)
            driver.get(url)
            
            lks = driver.find_elements_by_xpath('.//a[@class="tutor-card flex"]')
            lks_found = [link.get_attribute('href') for link in lks]
            links.append(lks_found)
            
            
            tot_tuts.append(driver.find_elements_by_tag_name('strong')[2].text) #per subject given zip
            
            time.sleep(np.random.rand()*5)
            
        time.sleep(np.random.rand()*5)
    
    unique_tuts = np.unique(np.array([item for sublist in links for item in sublist]))
    save_tut_id(unique_tuts)
    get_tutor_info(zip_code, unique_tuts, headers)
    tut_num_df = pd.DataFrame({'zip_code': [str(zip_code)]*12, 'tot_tuts': tot_tuts})
    tut_num_df.to_csv('/home/codonnell/tutors/tut_num_' +str(zip_code) + '.csv')
    
    time.sleep(np.random.rand()*5)

NameError: name 'driver' is not defined